In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")

In [3]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily =TavilySearchResults()

C:\Users\WIIS\AppData\Local\Temp\ipykernel_3772\1016542888.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily =TavilySearchResults()


In [6]:
tools=[arxiv,wiki,tavily]

In [7]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen/qwen3-32b")

In [8]:
llm.invoke("Who are you?")

AIMessage(content="<think>\nOkay, the user asked Who are you? I need to answer according to the instructions. First, I should clearly state my name, Qwen, and that I'm a large-scale language model developed by Alibaba Cloud's Tongyi Lab. Then, mention my capabilities, like answering questions, creating text such as stories, official documents, emails, and scripts, logical reasoning, programming, and more. Highlight that I support multiple languages. Finally, invite the user to ask questions or request assistance. Keep it natural, friendly, concise, and avoid markdown formatting. Let me put that together.\n</think>\n\nHello! I'm Qwen, a large-scale language model developed by Alibaba Cloud's Tongyi Lab. I can help you with answering questions, writing stories, official documents, emails, scripts, logical reasoning, programming, and more. I support multiple languages and aim to provide accurate and helpful responses. How can I assist you today? Feel free to ask anything! 😊", additional_k

In [9]:
llm_with_tools=llm.bind_tools(tools=tools)

In [10]:
llm_with_tools.invoke("what is the recent news in ai?")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for recent news in AI. I need to figure out which tool to use here. Let me check the available functions.\n\nFirst, there\'s the arxiv function, which is for querying arXiv papers. That\'s more for academic research, not news. Then there\'s Wikipedia, which is good for general information but might not have the latest updates. The third option is tavily_search_results_json, which is a search engine optimized for current events. Since the user is asking about recent news, the tavily search seems appropriate. \n\nI should make sure that the query is properly formatted. The function requires a "query" parameter. The user\'s question is straightforward, so the search query should be something like "recent news in AI". I\'ll use that as the argument for the tavily_search function. No need to check other functions here because the focus is on current events. Alright, let\'s go with that.\n', 'tool_calls':

In [11]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages 

In [12]:
class state(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]


In [13]:
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [14]:
#Node Defintition
def tool_calling_llm(state:state):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

# Build Graph
builder=StateGraph(state)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

In [15]:
# Edges
builder.add_edge(START,"tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge("tools",tool_calling_llm)

In [16]:
graph=builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Found edge ending at unknown node `<function tool_calling_llm at 0x000001766A4A7F60>`

In [17]:
messages=graph.invoke({"messages":"give latest news in ai and then give about attention is all needed research paper"})
for m in messages["messages"]:
    m.pretty_print()

NameError: name 'graph' is not defined